In [1]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import numpy as np
import pickle
import os
import torch.nn.functional as F
from PIL import Image
import random
import time
import torchvision

cuda_available = torch.cuda.is_available()

In [1]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Linear(10,5)
    def forward(self,x):
        return self.conv(x)

mynet = Net()
print(list(mynet.parameters()))

NameError: name 'nn' is not defined

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

class Net2(nn.Module):
    def __init__(self):
        super().__init__()        
        self.conv = nn.Conv2d(10,10,3)
        self.bn = nn.BatchNorm2d(10)
    def weight_init(self):
        for module in self.modules():
            if isinstance(module,nn.Conv2d):
                nn.init.normal_(module.weight,mean = 0,std = 1)
                nn.init.constant_(module.bias,0)
net2 = Net2()
net2.weight_init()
for module in net2.modules():
    if isinstance(module,nn.Conv2d):
        weights = module.weight
#         print(weights)
        weights = weights.flatten().detach().cpu().numpy()
#         print(weights.shape)
        print(module.bias)                                       # Bias to zero
        plt.hist(weights)
        plt.show()

NameError: name 'nn' is not defined

In [2]:
class ResBlock(nn.Module):
    expansion = 1
    def __init__(self,in_channels,out_channels,stride = 1):
        super(ResBlock,self).__init__()
        #conv1
        self.conv1 = nn.Conv2d(in_channels,out_channels,(3,3),stride = stride,padding = 1,bias = False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,out_channels,(3,3),stride = stride,padding = 1,bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        #Need to downsample if in_channels != out_channels
        if stride != 1 or in_channels!=out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,(1,1),stride = stride,bias = False),
                nn.BatchNorm2d(out_channels)
            )
        def forward(self,x):
            out = F.relu(self.bn(self.conv1(x)))
            out = self.bn2(self.conv2(out))
            out += self.shortcut(x)
            out = F.relu(out)
            return out

In [3]:
class ResNet(nn.Module):
    def __init__(self,num_classes = 10):
        super(ResNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64,kernel_size = (3,3),stride = 1,padding = 1,bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.block1 = self._create_block(64,64,stride = 1)
        self.block2 = self._create_block(64,128,stride = 2)
        self.block3 = self._create_block(128,256,stride = 2)
        self.block4 = self._create_block(256,512,stride = 2)
        self.linear = nn.Linear(512,num_classes)
        
    def _create_block(self,in_channels,out_channels,stride = 1):
        return nn.Sequential((ResBlock(in_channels,out_channels,stride)),
                            ResBlock(in_channels,out_channels,1))
    def forward(self, x):
        out = nn.ReLU()(self.bn1(self.conv1(x)))
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = nn.AvgPool2d(4)(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out    

In [4]:
from torchvision import datasets,transforms
data_transform = transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize(mean = [0.4914, 0.4822, 0.4465],std = [0.2023, 0.1994, 0.2010])])


In [5]:
dataset = datasets.CIFAR10('.',train = True,transform = data_transform,download = True)

Files already downloaded and verified


In [6]:
datatest = datasets.CIFAR10('.',train = False,transform = data_transform)

In [7]:
loadtrain = torch.utils.data.DataLoader(
    dataset, batch_size=128, shuffle=True, num_workers=2)
loadtest = torch.utils.data.DataLoader(
    dataset, batch_size=128, shuffle=False, num_workers=2)

In [9]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

False


In [24]:
for i in loadtrain:
    a,b = 
    break

[tensor([[[[-1.4017e+00, -1.5374e+00, -1.4986e+00,  ..., -2.2158e+00,
           -2.2352e+00, -2.2934e+00],
          [-1.2854e+00, -1.4211e+00, -1.4211e+00,  ..., -2.1383e+00,
           -2.1964e+00, -2.2934e+00],
          [-1.0915e+00, -1.1497e+00, -1.2078e+00,  ..., -1.9251e+00,
           -1.9638e+00, -2.0026e+00],
          ...,
          [ 3.0421e-01,  4.0113e-01,  4.9806e-01,  ...,  4.7867e-01,
            3.8175e-01,  2.2667e-01],
          [ 4.5929e-01,  4.9806e-01,  5.3683e-01,  ...,  6.9191e-01,
            5.7560e-01,  4.2052e-01],
          [ 3.8175e-01,  4.5929e-01,  5.1744e-01,  ...,  7.6945e-01,
            6.9191e-01,  5.7560e-01]],

         [[-1.9266e+00, -2.0446e+00, -2.0249e+00,  ..., -2.2216e+00,
           -2.2609e+00, -2.3003e+00],
          [-1.8873e+00, -1.9856e+00, -1.9856e+00,  ..., -2.1823e+00,
           -2.2609e+00, -2.3199e+00],
          [-1.8282e+00, -1.8676e+00, -1.8873e+00,  ..., -2.1233e+00,
           -2.1626e+00, -2.1823e+00],
          ...,
    

In [ ]:
clf = ResNet()
if cuda_available:
    clf = clf.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(clf.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 200], gamma=0.1)